## Importing packages for analysis

In [1]:
# Exploratory Data Analysis
import pandas as pd
import numpy as np
from collections import Counter

#Importing Apriori algorithm
from apyori import apriori

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report

# Settings
import warnings
warnings.filterwarnings("ignore")
import gc

## Data Preprocessing

In [2]:
# Loading and previewing data
df = pd.read_csv("fraudTrain.csv", engine='python')[:100000]
df.head(10)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
5,5,2019-01-01 00:04:08,4767265376804500,"fraud_Stroman, Hudson and Erdman",gas_transport,94.63,Jennifer,Conner,F,4655 David Island,...,40.3750,-75.2045,2158,Transport planner,1961-06-19,189a841a0a8ba03058526bcfe566aab5,1325376248,40.653382,-76.152667,0
6,6,2019-01-01 00:04:42,30074693890476,fraud_Rowe-Vandervort,grocery_net,44.54,Kelsey,Richards,F,889 Sarah Station Suite 624,...,37.9931,-100.9893,2691,Arboriculturist,1993-08-16,83ec1cc84142af6e2acf10c44949e720,1325376282,37.162705,-100.153370,0
7,7,2019-01-01 00:05:08,6011360759745864,fraud_Corwin-Collins,gas_transport,71.65,Steven,Williams,M,231 Flores Pass Suite 720,...,38.8432,-78.6003,6018,"Designer, multimedia",1947-08-21,6d294ed2cc447d2c71c7171a3d54967c,1325376308,38.948089,-78.540296,0
8,8,2019-01-01 00:05:18,4922710831011201,fraud_Herzog Ltd,misc_pos,4.27,Heather,Chase,F,6888 Hicks Stream Suite 954,...,40.3359,-79.6607,1472,Public affairs consultant,1941-03-07,fc28024ce480f8ef21a32d64c93a29f5,1325376318,40.351813,-79.958146,0
9,9,2019-01-01 00:06:01,2720830304681674,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,198.39,Melissa,Aguilar,F,21326 Taylor Squares Suite 708,...,36.5220,-87.3490,151785,Pathologist,1974-03-28,3b9014ea8fb80bd65de0b1463b00b00e,1325376361,37.179198,-87.485381,0


In [3]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05,1.000000e+05,100000.000000,100000.000000,100000.000000
mean,49999.500000,4.183992e+17,71.908232,48720.506960,38.538809,-90.179763,8.918773e+04,1.327916e+09,38.539719,-90.180833,0.009900
std,28867.657797,1.309964e+18,145.895400,26895.490176,5.077738,13.771942,3.015788e+05,1.459296e+06,5.110508,13.783471,0.099005
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.325376e+09,19.029798,-166.670132,0.000000
25%,24999.750000,1.800365e+14,9.710000,26041.000000,34.668900,-96.790900,7.430000e+02,1.326627e+09,34.750439,-96.867861,0.000000
50%,49999.500000,3.519607e+15,48.150000,48174.000000,39.354300,-87.458100,2.456000e+03,1.327877e+09,39.365135,-87.407044,0.000000
75%,74999.250000,4.642255e+15,83.870000,72011.000000,41.894800,-80.128400,2.047800e+04,1.329150e+09,41.943393,-80.180297,0.000000
max,99999.000000,4.992346e+18,12788.070000,99783.000000,65.689900,-67.950300,2.906700e+06,1.330444e+09,66.659242,-66.967742,1.000000


## Sanity Check

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             100000 non-null  int64  
 1   trans_date_trans_time  100000 non-null  object 
 2   cc_num                 100000 non-null  int64  
 3   merchant               100000 non-null  object 
 4   category               100000 non-null  object 
 5   amt                    100000 non-null  float64
 6   first                  100000 non-null  object 
 7   last                   100000 non-null  object 
 8   gender                 100000 non-null  object 
 9   street                 100000 non-null  object 
 10  city                   100000 non-null  object 
 11  state                  100000 non-null  object 
 12  zip                    100000 non-null  int64  
 13  lat                    100000 non-null  float64
 14  long                   100000 non-nul

In [5]:
#Dropping of redundant attributes
df = df.drop(labels = ['Unnamed: 0', 'trans_date_trans_time','street', 'first', 'last', 'city', 'zip', 'trans_num', 'dob'], axis = 1) 

In [6]:
df

,cc_num,merchant,category,amt,gender,state,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,NC,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,36.011293,-82.048315,0
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,WA,48.8878,-118.2105,149,Special educational needs teacher,1325376044,49.159047,-118.186462,0
2,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,ID,42.1808,-112.2620,4154,Nature conservation officer,1325376051,43.150704,-112.154481,0
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,MT,46.2306,-112.1138,1939,Patent attorney,1325376076,47.034331,-112.561071,0
4,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,VA,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2475085306462014,"fraud_O'Reilly, Mohr and Purdy",home,95.14,M,MN,44.2378,-95.2739,1507,Land/geomatics surveyor,1330443409,44.718105,-95.843397,0
99996,4005676619255478,fraud_Kub PLC,personal_care,8.75,M,LA,30.4590,-90.9027,71335,Herbalist,1330443447,29.931844,-90.610715,0
99997,3519232971341141,fraud_Schuppe-Schuppe,food_dining,34.20,M,OH,40.4731,-80.9596,2208,Mental health nurse,1330443454,41.076153,-80.506107,0
99998,4040099974063068803,fraud_Rippin-VonRueden,health_fitness,73.11,M,ND,48.3396,-102.2400,229,Administrator,1330443491,48.535070,-102.524262,0


In [7]:
#Helper functions to preprocess data

# Function: discretize(df)
# Input: Dataframe
# Return output: Dataframe of continuous attributes converted to categorical data
# Function Description: 
#   - Converts amt, lat, long, city_pop, unix_time, merch_lat, merch_long to categorical data

def discretize(dataframe):
    dataframe['amt'] = pd.cut(dataframe['amt'], bins=20, right=False)
    dataframe['lat'] = pd.cut(dataframe['lat'], bins=10, right=False)
    dataframe['long'] = pd.cut(dataframe['long'], bins=10, right=False)
    dataframe['city_pop'] = pd.cut(dataframe['city_pop'], bins=10, right=False)
    dataframe['unix_time'] = pd.cut(dataframe['unix_time'], bins=10, right=False)
    dataframe['merch_lat'] = pd.cut(dataframe['merch_lat'], bins=10, right=False)
    dataframe['merch_long'] = pd.cut(dataframe['merch_long'], bins=10, right=False)
    return dataframe

# Function: convert_to_str(df)
# Input: Dataframe
# Return output: Dataframe 
# Function Description: 
#   - Converts all variables in dataframe to string
def convert_to_str(dataframe):
    dataframe = dataframe.astype(str)
    return dataframe

In [8]:
#Preparing dataset without discretizing
df_wo_discretize = df 
df_wo_discretize = convert_to_str(df_wo_discretize)
df_wo_discretize

,cc_num,merchant,category,amt,gender,state,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,NC,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,36.011293,-82.048315,0
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,WA,48.8878,-118.2105,149,Special educational needs teacher,1325376044,49.159047,-118.186462,0
2,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,ID,42.1808,-112.262,4154,Nature conservation officer,1325376051,43.150704,-112.154481,0
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.0,M,MT,46.2306,-112.1138,1939,Patent attorney,1325376076,47.034331,-112.561071,0
4,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,VA,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2475085306462014,"fraud_O'Reilly, Mohr and Purdy",home,95.14,M,MN,44.2378,-95.2739,1507,Land/geomatics surveyor,1330443409,44.718105,-95.843397,0
99996,4005676619255478,fraud_Kub PLC,personal_care,8.75,M,LA,30.459,-90.9027,71335,Herbalist,1330443447,29.931844,-90.610715,0
99997,3519232971341141,fraud_Schuppe-Schuppe,food_dining,34.2,M,OH,40.4731,-80.9596,2208,Mental health nurse,1330443454,41.07615300000001,-80.506107,0
99998,4040099974063068803,fraud_Rippin-VonRueden,health_fitness,73.11,M,ND,48.3396,-102.24,229,Administrator,1330443491,48.53507000000001,-102.524262,0


In [9]:
#Preparing dataset with discretizing
df_discretize = discretize(df)
df_discretize = convert_to_str(df_discretize)
df_discretize

,cc_num,merchant,category,amt,gender,state,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,"[1.0, 640.353)",F,NC,"[33.726, 38.292)","[-87.495, -77.722)","[23.0, 290690.7)","Psychologist, counselling","[1325376018.0, 1325882778.2)","[33.319, 38.082)","[-86.908, -76.938)",0
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,"[1.0, 640.353)",F,WA,"[47.425, 51.991)","[-126.584, -116.811)","[23.0, 290690.7)",Special educational needs teacher,"[1325376018.0, 1325882778.2)","[47.607, 52.37)","[-126.789, -116.819)",0
2,38859492057661,fraud_Lind-Buckridge,entertainment,"[1.0, 640.353)",M,ID,"[38.292, 42.858)","[-116.811, -107.039)","[23.0, 290690.7)",Nature conservation officer,"[1325376018.0, 1325882778.2)","[42.845, 47.607)","[-116.819, -106.849)",0
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,"[1.0, 640.353)",M,MT,"[42.858, 47.425)","[-116.811, -107.039)","[23.0, 290690.7)",Patent attorney,"[1325376018.0, 1325882778.2)","[42.845, 47.607)","[-116.819, -106.849)",0
4,375534208663984,fraud_Keeling-Crist,misc_pos,"[1.0, 640.353)",M,VA,"[38.292, 42.858)","[-87.495, -77.722)","[23.0, 290690.7)",Dance movement psychotherapist,"[1325376018.0, 1325882778.2)","[38.082, 42.845)","[-86.908, -76.938)",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2475085306462014,"fraud_O'Reilly, Mohr and Purdy",home,"[1.0, 640.353)",M,MN,"[42.858, 47.425)","[-97.267, -87.495)","[23.0, 290690.7)",Land/geomatics surveyor,"[1329936859.8, 1330448687.602)","[42.845, 47.607)","[-96.878, -86.908)",0
99996,4005676619255478,fraud_Kub PLC,personal_care,"[1.0, 640.353)",M,LA,"[29.16, 33.726)","[-97.267, -87.495)","[23.0, 290690.7)",Herbalist,"[1329936859.8, 1330448687.602)","[28.556, 33.319)","[-96.878, -86.908)",0
99997,3519232971341141,fraud_Schuppe-Schuppe,food_dining,"[1.0, 640.353)",M,OH,"[38.292, 42.858)","[-87.495, -77.722)","[23.0, 290690.7)",Mental health nurse,"[1329936859.8, 1330448687.602)","[38.082, 42.845)","[-86.908, -76.938)",0
99998,4040099974063068803,fraud_Rippin-VonRueden,health_fitness,"[1.0, 640.353)",M,ND,"[47.425, 51.991)","[-107.039, -97.267)","[23.0, 290690.7)",Administrator,"[1329936859.8, 1330448687.602)","[47.607, 52.37)","[-106.849, -96.878)",0


In [10]:
#Checking of bins for amt
Counter(df.amt)

Counter({Interval(1.0, 640.353, closed='left'): 99039,
         Interval(640.353, 1279.707, closed='left'): 830,
         Interval(1279.707, 1919.06, closed='left'): 81,
         Interval(2558.414, 3197.768, closed='left'): 9,
         Interval(1919.06, 2558.414, closed='left'): 24,
         Interval(5115.828, 5755.182, closed='left'): 2,
         Interval(11509.363, 12148.716, closed='left'): 1,
         Interval(5755.182, 6394.535, closed='left'): 2,
         Interval(3197.768, 3837.121, closed='left'): 8,
         Interval(10230.656, 10870.009, closed='left'): 1,
         Interval(4476.474, 5115.828, closed='left'): 1,
         Interval(12148.716, 12800.857, closed='left'): 1,
         Interval(3837.121, 4476.474, closed='left'): 1})

## Preparing of Train and Test sets

In [11]:
df_discretize.shape

(100000, 14)

In [12]:
#Splitting of train, test, validation set for dataset with discretization
train_val, test = train_test_split(df_discretize, test_size=0.10,random_state=42) #For testing
train, val = train_test_split(train_val, test_size=0.20,random_state=42) #For validation

#Splitting of train, test, validation set for dataset without discretization
train_val_wo_discretize, test_wo_discretize = train_test_split(df_wo_discretize, test_size=0.10,random_state=42) #For testing
train_wo_discretize, val_wo_discretize = train_test_split(train_val_wo_discretize, test_size=0.20,random_state=42) #For validation

## Applying FraudMiner model

In [13]:
#Helper functions to apply FraudMiner model

# Function: max_apriori(FIS)
# Input: Frequent Item Sets
# Return output: Largest Item Set 
# Function Description: 
#   - Takes in a list of frequent item sets and returns the largest itemset in the list
def max_apriori(FIS):
    results = list(FIS)
    max_frozen_set = frozenset({})
    for i in range(0, len(results)):
        frozen_set = results[i][0]
        if len(frozen_set) > len(max_frozen_set):
            max_frozen_set = frozen_set

    return max_frozen_set

# Function: training_algorithm_legal(df, support)
# Input: Dataframe and support level
# Return output: List of itemsets that represents the legal pattern of each customer
# Function Description: 
#   - Takes in a dataframe of customer transactions 
#   - For each customer, it separates a customer transactions from the rest of the dataframe
#   - Separates the legal transactions from the fraud transaction of the particular customer
#   - Finds the frequent itemset of the customer using Apriori algorithm and then finds the largest itemset using max_apriori function
#   - Stores the largest itemset in a list and returns the list
def training_algorithm_legal(train_df, support):
    uniqueCreditCard = train_df['cc_num'].unique()
 
    LPD = []
    for i in uniqueCreditCard: 
        #Transactions of a user
        customer_transactions = train_df[train_df.cc_num == i]   
        #Legal transactions
        legal = customer_transactions[customer_transactions.is_fraud == '0'] 
        
        #Converting the legal transactions into a list of list
        legal_transactions = []
        for i in range(0, legal.shape[0]):  
            legal_transactions.append([legal.values[i,u] for u in range(0, 13)])
       
        #Finding max frequent set for legal
        FIS = apriori(legal_transactions, min_support = support, use_colnames=True)
        max_frequent_set = max_apriori(FIS)
        LPD.append(max_frequent_set)
    return LPD

# Function: training_algorithm_fraud(df, support)
# Input: Dataframe and support level
# Return output: List of itemsets that represents the fraud pattern of each customer
# Function Description: 
#   - Takes in a dataframe of customer transactions 
#   - For each customer, it separates a customer transactions from the rest of the dataframe
#   - Separates the fraud transactions from the legal transaction of the particular customer
#   - Finds the frequent itemset of the customer using Apriori algorithm and then finds the largest itemset using max_apriori function
#   - Stores the largest itemset in a list and returns the list
def training_algorithm_fraud(train_df, support):
    uniqueCreditCard = train_df['cc_num'].unique()
 
    FPD = []
    for i in uniqueCreditCard: 
        
        #Transactions of a user
        customer_transactions = train_df[train_df.cc_num == i]   
        #Legal transactions
        fraud = customer_transactions[customer_transactions.is_fraud == '1'] 
        
        #Converting the legal transactions into a list of list
        fraud_transactions = []
        for i in range(0, fraud.shape[0]):  
            fraud_transactions.append([fraud.values[i,u] for u in range(0, 13)])
        
        #Finding max frequent set for legal
        FIS = apriori(fraud_transactions, min_support = support, use_colnames=True)
        max_frequent_set = max_apriori(FIS)
        FPD.append(max_frequent_set)
    return FPD

# Function: testing_algorithm(LPD, FPD, incomingTransactionDataframe, noOfCustomers, noOfAttributes, matchingPercentage)
# Input: 
# Return output: List of predictions
# Function Description: 
#   - Compares the incoming transactions with the legal and fraud pattern databases, and uses a score (lc and fc) to
#     determine whether the incoming transaction is fraud or not
def testing_algorithm(LPD, FPD, incomingTransactionDataframe, noOfCustomers, noOfAttributes, matchingPercentage):
    predictions = []
    for i in range(len(incomingTransactionDataframe)):
        incomingTransaction = list(incomingTransactionDataframe.iloc[i])
        lc = 0      #legal attribute match count
        fc = 0      #fraud attribute match count
        for i in range(noOfCustomers):
            if incomingTransaction[0] in list(LPD[i]):  #Matching of customer id
                for attribute in incomingTransaction:
                    if attribute in list(LPD[i]):
                        lc += 1
        for i in range(noOfCustomers):
            if incomingTransaction[0] in list(FPD[i]):  #Matching of customer id
                for attribute in incomingTransaction:
                    if attribute in list(FPD[i]):
                        fc += 1

        if fc == 0:     #no fraud pattern
            if lc/noOfAttributes >= matchingPercentage:
                predictions.append(0)
            else:
                predictions.append(1)
        elif lc == 0:     #no legal pattern
            if fc/noOfAttributes >= matchingPercentage:
                predictions.append(1)
            else:
                predictions.append(0)
        else:     #has both legal and fraud pattern
            if fc >= lc:
                predictions.append(1)
            else: 
                predictions.append(0)
    return predictions

def parameter_tuning(train, LPD, FPD, val): 
    percentage_grid = [x/20 for x in list(range(0,21))]
    noOfCustomers = train['cc_num'].nunique()
    noOfAttributes = train.shape[1] - 1
    store = {}

    for i in percentage_grid:
        res = {}
        matchingPercentage = i
        predictions = testing_algorithm(LPD, FPD, val, noOfCustomers, noOfAttributes, matchingPercentage)
        y_true = val['is_fraud']
        y_true_int, y_pred_int = [int(j) for j in y_true],[int(i) for i in predictions]
        tn, fp, fn, tp = confusion_matrix(y_true_int, y_pred_int).ravel()
        res["tn"] = tn
        res["fp"] = fp
        res["fn"] = fn
        res["tp"] = tp
        res["recall"] = recall_score(y_true_int, y_pred_int)
        res["precision_score"] = precision_score(y_true_int, y_pred_int)
        res["f1_score"] = f1_score(y_true_int, y_pred_int)
        res["sensitivity"] = tp/(tp+fn)
        store[i] = res
    df_res = pd.DataFrame(store).T
    return df_res

In [14]:
#Obtaining pattern databases

#For dataset with discretization
LPD = training_algorithm_legal(train, 0.9)
FPD = training_algorithm_fraud(train, 0.9)

#For dataset without discretization
LPD_wo_discretize = training_algorithm_legal(train_wo_discretize, 0.9)
FPD_wo_discretize = training_algorithm_fraud(train_wo_discretize, 0.9)

## Conducting hyperparameter tuning for matching percentage

In [15]:
parameter_tuning(train, LPD, FPD, val)

,tn,fp,fn,tp,recall,precision_score,f1_score,sensitivity
0.00,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.05,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.10,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.15,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.20,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.25,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.30,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.35,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.40,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500
0.45,17493.0,347.0,12.0,148.0,0.92500,0.298990,0.451908,0.92500


As seen from the table, matching percentage of 0.5 will give the best recall and sensitivity score. Hence, a matching percentage of 0.5 will be selected. 

## Evaluation of model

For dataset with discretization

In [16]:
#Running predictions
noOfCustomers = train['cc_num'].nunique()
noOfAttributes = train.shape[1] - 1
matchingPercentage = 0.5 # optimal value of matchingPercentage as determined above
predictions = testing_algorithm(LPD, FPD, test, noOfCustomers, noOfAttributes, matchingPercentage)

In [17]:
#Printing classification report
y_true = test['is_fraud']
print(classification_report([int(j) for j in y_true],[int(i) for i in predictions]))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      9900
           1       0.37      0.93      0.53       100

    accuracy                           0.98     10000
   macro avg       0.68      0.96      0.76     10000
weighted avg       0.99      0.98      0.99     10000



In [18]:
#Printing confusion matrix
y_true_int, y_pred_int = [int(j) for j in y_true],[int(i) for i in predictions]
tn, fp, fn, tp = confusion_matrix(y_true_int, y_pred_int).ravel()
print([tn,fp,fn,tp])

[9740, 160, 7, 93]


In [19]:
#Metrics calculation
recall = recall_score(y_true_int, y_pred_int)
precision = precision_score(y_true_int, y_pred_int)
f2_score = (5*precision*recall)/(4*precision + recall)

In [20]:
#Printing results in table
results_dict = {}
score = {
    "recall": recall,
    "precision":precision,
    "f2 measure": f2_score
}

results_dict['FraudMiner'] = score
pd.DataFrame(results_dict)

,FraudMiner
f2 measure,0.712098
precision,0.367589
recall,0.930000


For dataset without discretization

In [21]:
#Running predictions
predictions_wo_discretize = testing_algorithm(LPD_wo_discretize, FPD_wo_discretize, test_wo_discretize, noOfCustomers, noOfAttributes, matchingPercentage)

In [22]:
#Printing classification report
y_true_wo_discretize = test_wo_discretize['is_fraud']
print(classification_report([int(j) for j in y_true_wo_discretize],[int(i) for i in predictions_wo_discretize]))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95      9900
           1       0.09      1.00      0.17       100

    accuracy                           0.90     10000
   macro avg       0.55      0.95      0.56     10000
weighted avg       0.99      0.90      0.94     10000



In [23]:
#Printing confusion matrix
y_true_int_2, y_pred_int_2 = [int(j) for j in y_true_wo_discretize],[int(i) for i in predictions_wo_discretize]
tn_2, fp_2, fn_2, tp_2 = confusion_matrix(y_true_int_2, y_pred_int_2).ravel()
print([tn_2,fp_2,fn_2,tp_2])

[8934, 966, 0, 100]


In [24]:
#Metrics calculation
recall_2 = recall_score(y_true_int_2, y_pred_int_2)
precision_2 = precision_score(y_true_int_2, y_pred_int_2)
f2_score_2 = (5*precision_2*recall_2)/(4*precision_2 + recall_2)

In [25]:
#Printing results in table
results_dict_2 = {}
score_2 = {
    "recall": recall_2,
    "precision":precision_2,
    "f2 measure": f2_score_2
}

results_dict_2['FraudMiner without discretization'] = score_2
pd.DataFrame(results_dict_2)

,FraudMiner without discretization
f2 measure,0.341064
precision,0.093809
recall,1.000000


## Comparison between FraudMiner with discretization and without discretization

In [26]:
pd.DataFrame({'FraudMiner with discretization':pd.Series(score),'FraudMiner without discretization':pd.Series(score_2)})

,FraudMiner with discretization,FraudMiner without discretization
recall,0.930000,1.000000
precision,0.367589,0.093809
f2 measure,0.712098,0.341064


As seen from above, FraudMiner with discretization performs much better in terms of precision than without discretization. Hence, we will be choosing to conduct discretization as low precision means that the model is returning flagging too many non-fraud transactions as fraud, and this could lead to customer dissatisfaction.